# Machine Learning - Assignment 3

### Oscar Fabián Ñáñez Núñez - Juan Camilo Calero Espinosa
### ofnanezn - jccaleroe

In [1]:
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import scale
import sklearn
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as AL
from sklearn.kernel_approximation import RBFSampler
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from threading import Thread
from time import time
from sklearn.metrics import accuracy_score


## 3)

## n-grams kernel 

In [2]:
def ngk(doc1, doc2, n=2, mode='char'):
    """ n: Order of the n-gram
    mode: Either 'char' or 'word' """

    # Counts the occurences of unique n-grams
    ngrams = CountVectorizer(analyzer=mode, ngram_range=(n, n)).fit_transform([doc1, doc2])

    # Normalize
    #a, b = TfidfTransformer().fit_transform(ngrams).toarray()
    a, b = ngrams.toarray()
    return np.dot(a, b)

In [3]:
english = np.genfromtxt('words/english400.txt',dtype='str')
spanish = np.genfromtxt('words/spanish400.txt',dtype='str')
sl = np.zeros(spanish.shape)
el = np.full(spanish.shape, 1)
X = np.concatenate((spanish, english), axis=0)
Y = np.concatenate((sl, el), axis=0)

englishTest = np.genfromtxt('words/englishTest100.txt',dtype='str')
spanishTest = np.genfromtxt('words/spanishTest100.txt',dtype='str')
slt = np.zeros(spanishTest.shape)
elt = np.full(spanishTest.shape, 1)
XT = np.concatenate((spanishTest, englishTest), axis=0)
YT = np.concatenate((slt, elt), axis=0)

In [4]:
def ngkClassifier(n):
    #t = time()
    gram2 = np.zeros((X.shape[0], X.shape[0]))
    for i in range(X.shape[0]):
        for j in range(X.shape[0]):
            gram2[i][j] = ngk(X[i], X[j], n)

    #print("gram2 preprocessed")
    #print(round(time()-t))

    clf2 = svm.SVC(kernel='precomputed')
    clf2.fit(gram2, Y)

    #t = time()
    gramT2 = np.zeros((XT.shape[0], X.shape[0]))
    for i in range(XT.shape[0]):
        for j in range(X.shape[0]):
            gramT2[i][j] = ngk(XT[i], X[j], n)

    #print("test gram2 preprocessed")
    #print(round(time()-t))

    Z2 = clf2.predict(gramT2)
    #print(Z2)
    #print(YT)
    #print(XT)
    a = accuracy_score(YT, Z2)
    print(a)
    return a

In [5]:
nMax, tmp = 0, 0
for n in range(1, 5):
    print("n ", n)
    aux = ngkClassifier(n)
    if aux > tmp:
        tmp = aux
        nMax = n
        
print("Max n:", nMax, "value:", tmp)


n  1
0.715
n  2
0.785
n  3
0.71
n  4
0.695
Max n: 2 value: 0.785


### Best parameter: 
### n = 2